# preparation

In [ ]:
!pip install bing-image-downloader

In [ ]:
!mkdir Dataset

In [ ]:
from bing_image_downloader import downloader as dl

nd = ["Gambar Banjir","Gambar Gempa Bumi","Gambar Tsunami","Gambar tanah longsor"]
dl.download(nd[0],limit=500,output_dir='Dataset')
dl.download(nd[1],limit=500,output_dir='Dataset')
dl.download(nd[2],limit=500,output_dir='Dataset')
dl.download(nd[3],limit=500,output_dir='Dataset')

In [ ]:
dl.download(nd[1],limit=1000,output_dir='Dataset')


In [ ]:
dl.download(nd[2],limit=1000,output_dir='Dataset')


In [ ]:
dl.download(nd[3],limit=1000,output_dir='Dataset')


# Model Building

## lib

In [11]:
# common lib
import os , shutil
import zipfile
import random
import pathlib
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample
from shutil import copyfile
from pathlib import Path
from tqdm.notebook import tqdm as tq
warnings.simplefilter(action='ignore', category=FutureWarning)
from google.colab import files

# image lib
import cv2
import skimage
from PIL import Image
from skimage import io
from skimage.transform import resize
from skimage.transform import rotate, AffineTransform, warp
from skimage import img_as_ubyte
from skimage.exposure import adjust_gamma
from skimage.util import random_noise

# model creation
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import Model, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import InputLayer, Conv2D, SeparableConv2D, MaxPooling2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model


## starting